# DWA-A102 library

**Author: Edwin Echeverri S. (2021-03-27)**


## Import

In [1]:
# import standard libraries
import numpy as np
import pandas as pd
from IPython.display import Image

# import dwa-a102 libraries
from dwa_a102 import StudyArea as StudyArea
from dwa_a102 import watbal as watbal
from check_ranges import validRange
from climate import climate

## 1. Introduction

This notebook presents some basic examples of the usage of the library DWA-A102. The purpose of the deloved code is to provide a tool to calculate the main components of the water balance in urban areas. The applied equations are the presented in the German Guideline DWA-A102 (2017). The balance is considered as:

<center>$P_{korr} = R_D + ET_a+ GWN $<center>

That can be rewriten as:

<center>$P_{korr} = a*P_{korr} + v*P_{korr} + g*P_{korr}$<center>

Having:

<center>$1 = a + v + g$<center>
<br \n>
    
<center>(DWA-A 102, 2017)<center>

$P_{korr}$	: Precipitation (corrected) (mm/a), 
$R_D$	: Surface runoff (mm/a), 
$ET_a$	: Actual evapotranspiration (mm/a), 
$GWN$	: Groundwater recharge (mm/a), 
$a$	: Partitioning factor for surface runoff (-), 
$v$	: Partitioning factor for evapotranspiration (-), 
$g$	: Partitioning factor for groundwater recharge (-).

The structure of the library is presented in the following figure:

<br></br>
<center><img src="images/dwa-a102_Structure.png"  style="width: 600px"><center>
<br></br>
    
<center> <b> Fig. 1: Classes and functions of package </b> <center>

The Classes Surface() and Measure() contains the following methods:

<br></br>
<center><img src="images/dwa-a102_Methods.png"  style="width: 600px"><center>
<br></br>
    
<center> <b> Fig. 2: Methods of classes "Surface" and "Measure" </b> <center>

## 2. Examples

### 2.1 Create Study Area
The first recommended step is to define your Study Area with the climatic data (P, ETp). By default, the library considers a P=800 mm/a, and ETp = 500 mm/a.

In [2]:
my_area = StudyArea(p=900, etp=600)
print(my_area)

Study area has a precipitation of 900 mm/a, and potential evapotranspiration of 600 mm/a


In [3]:
my_area2 = StudyArea()
print(my_area2)

Study area has a precipitation of 800 mm/a, and potential evapotranspiration of 500 mm/a


The user can use the reference climatic values that the package contians for the 58 biggest cities in Germany. In this case, it is only necessary to define the location as follows:

In [4]:
hannover = StudyArea(location="Hannover")
print(hannover)

Study area has a precipitation of 700 mm/a, and potential evapotranspiration of 575 mm/a


In either case, whether the user enters the climatic values or uses the reference dictionary, the values are validated to see if they are within the range of validity of the equations.

In [5]:
my_area3 = StudyArea(p=1750, etp=400)

Exception: Precipitation is not valid. Acceptable range: 500 - 1700 mm/a

In [6]:
my_area3 = StudyArea(p=1700, etp=400)

Exception: Evapotranspiration is not valid. Acceptable range: 450 - 700 mm/a

In [7]:
my_area3 = StudyArea(p=1700, etp=450)
print(my_area3)

Study area has a precipitation of 1700 mm/a, and potential evapotranspiration of 450 mm/a


## 2.2 Surfaces
Consider that in Hannover a building that is sorrounded by a parking area and some gardens as shown in the following figure.
    
<br></br>
<center><img src="images/LUH_Building_Example.png"  style="width: 450px"><center>
<br></br>

<center> <b> Fig. 3: Study area in Hannover </b> <center>
<center> Source: Google Earth <center>

The type of surfaces, their area, and method to use in the library are presented in the following table:

<center> <b> Table 1: Surfaces and methods to apply in example </b> <center>
      
    
| Surface     	| Area (m<sup>2</sup>) 	| Method to use   	                     |
|-------------	|-----------	        |-----------------	                     |
| Steep roof  	| 1100      	        | roof(self, area, sp=0.3)    	         |
| Parking lot 	| 1500      	        | flat_area(self, area, sp=1) 	         |
| Garden      	| 440       	        | garden(self, area, a=0.2, g=0.2, v=0.6)|
| Flat roof   	| 92        	        | roof(self, area, sp=0.6)       	     |
| Total area  	| 3132      	        |                 	                     |

In case the user requires details about the methods, the method "help" describes the method, as well details about inputs, default values, and outputs. To access to the help function just type "help(StudyArea(method)", for example "help(StudyArea(green_roof)". In this example, because "hannover" is already a StudyArea, we can call directly the helps function as:

In [8]:
help(hannover.roof)

Help on method roof in module dwa_a102:

roof(area, sp=0.3) method of dwa_a102.StudyArea instance
    Calculates water balance components for steep roofs (all materials)
    or flat roofs made with smooth materials (e.g. glass, metal)
    
    Parameters
    ----------
    Area : float
         element area (m2)    
           
    Sp : float
       storage height (mm)
       
    Notes    
    ------
    Ranges of validity for the parameters are:
      Sp: 0.1 - 0.6 mm
      
    Standard Sp-values are:
      Steep roof: Sp = 0.3 mm
      Flat with smooth cover (glass or metal): Sp = 0.6 mm
    
    Returns
    -------
    results : DataFrame



In [9]:
steep_roof = hannover.roof(area=1100, sp=0.3)
print(steep_roof)

  Element  Area   Au    P  Etp      a  g      v  e   Vp   Va  Vg  Vv  Ve
0    Roof  1100  993  700  575  0.903  0  0.097  0  770  695   0  75   0


With: <br />

Area: surface of element (m<sup>2</sup>) \
Au: effective area generating runoff (m<sup>2</sup>), (calculated as Area*a)\
P: precipitation (mm/a)\
ETp: potential evapotranspiration (mm/a)\
a	: partitioning factor for surface runoff (-)\
g: partitioning factor for groundwater recharge (-)\
v: partitioning factor for evapotranspiration (-)\
e: fractioning factor of precipitation for rainwater usage/harvesting (-)\
Vp: volume of precipitation in a year over the element (m<sup>3</sup>)\
Vg: volume of groundwater recharge in a year due to the element (m<sup>3</sup>)\
Vv: evapo(transpi)red volume in a year over the element (m<sup>3</sup>)\
Ve: volume of water used/harvested per year (m<sup>3</sup>).

In [10]:
parking_lot = hannover.flat_area(area=1500)
print(parking_lot)

     Element  Area    Au    P  Etp      a  g      v  e    Vp   Va  Vg   Vv  Ve
0  Flat area  1500  1227  700  575  0.818  0  0.182  0  1050  859   0  191   0


In [11]:
garden = hannover.garden(area=440)
print(garden)

               Element  Area  Au    P  Etp    a    g    v  e   Vp  Va  Vg  \
0  Garden / green area   440  88  700  575  0.2  0.2  0.6  0  308  62  62   

    Vv  Ve  
0  185   0  


In [12]:
flat_roof = hannover.roof(area=92, sp=0.6)
print(flat_roof)

  Element  Area  Au    P  Etp     a  g     v  e  Vp  Va  Vg  Vv  Ve
0    Roof    92  79  700  575  0.86  0  0.14  0  64  55   0   9   0


In this point, it is relevant to consider the whole water balance of system compound by the designed elements. In this step we use the function "watbal()" as follows: 

In [13]:
watbal(steep_roof, parking_lot, garden, flat_roof)

,Element,Area,a,g,v,Vp,Va,Vg,Vv
0,Roof,1100,0.903,0.000,0.097,770,695,0,75
1,Flat area,1500,0.818,0.000,0.182,1050,859,0,191
2,Garden / green area,440,0.200,0.200,0.600,308,62,62,185
3,Roof,92,0.860,0.000,0.140,64,55,0,9
4,System,3132,0.762,0.028,0.210,2192,1671,62,460


Notice that 76% of the precipitation becomes surface runoff. While the groundwater recharge is below 3%, and the losses due to evapotranspiration reach 21%.

## 2.3 Measures
The University´s Infrastructure Office wants to know how some changes in the type of surfaces and the installation of some stormwater measures contribute to restoring the natural water balance in a pilot area. The main ideas consist of changing the surface type of the parking lots from paved into grass pavers. The flat roof would be adequate as a green roof, and the installation of a swale to manage the runoff produced by the steep roof is considered. The final pilot area would look like the following figure:
    
<br></br>
<center><img src="images/LUH_Building_Example_&_Measures.png"  style="width: 450px"><center>
<br></br>
    
<center> <b> Fig. 4: Study area in Hannover after modifiying soil type</b> <center>
<center> Source: Adapted from Google Earth <center>
<br></br>    
 
<center> <b> Table 2: Surfaces, measures, and methods to be applied </b> <center>    
    
| Surface     	| New surface  	| Measure            	| Area (m<sup>2</sup>) 	| Method for   surface type                                 	| Method for   measure                                       	|
|-------------	|--------------	|--------------------	|-----------	|-----------------------------------------------------------	|------------------------------------------------------------	|
| Steep roof  	|             	| Infiltration swale 	| 1100      	| roof(self, area,   sp=0.3)                                	|  infilt_swale(self, kf, *surfaces,   fasm="fasm_standard") 	|
| Parking lot 	| Green pavers 	|                    	| 1500      	| paver_stonegrid(self,   area, fa=25, sp=1, wkmax_wp=0.15) 	|                                                            	|
| Garden      	|             	|                    	| 440       	| garden(self, area,   a=0.2, g=0.2, v=0.6)                 	|                                                            	|
| Flat roof   	| Green roof   	|                    	| 92        	| green_roof(self,   area, h, kf=70, wkmax_wp=0.5)          	|                                                            	|
| Total area  	|              	|                    	| 3132      	|                                                           	|                                                            	|

In [14]:
steep_roof = hannover.roof(area=1100, sp=0.3)
swale = hannover.infilt_swale(42, steep_roof)
print(swale)

         Element  Area   Au    P  Etp      a     g      v  e   Vp  Va   Vg  \
0           Roof  1100  993  700  575  0.903     0  0.097  0  770   0    0   
1  Infilt. swale   130  993  700  575  0.000  0.95  0.051  0   91   0  747   

   Vv Ve  
0  75  0  
1  40  0  


In [15]:
parking_lot = hannover.paver_stonegrid(1500)
garden = hannover.garden(area=310)
green_roof = hannover.green_roof(92, 100)

The water balance of the analyzed system after the installation of the measures is:

In [16]:
watbal(swale, parking_lot, garden, green_roof)

,Element,Area,a,g,v,Vp,Va,Vg,Vv
0,Roof,1100,0.903,0,0.097,770,0,0,75
1,Infilt. swale,130,0.000,0.95,0.051,91,0,747,40
2,Paver stone-grid,1500,0.044,0.72,0.236,1050,46,756,248
3,Garden / green area,310,0.200,0.2,0.600,217,43,43,130
4,Green foof,92,0.506,0,0.494,64,33,0,32
5,System,3132,0.056,0.705,0.240,2192,122,1546,525


The water balance of the pilot area changes considerably. The surface runoff is around 5%, while the groundwater recharge represents 71%, and only 24% of the losses correspond to evapotranspiration. At first glance, the result seems adequate. However, the current configuration may produce an excessive (non-natural) groundwater recharge rate. The measures to be applied should increase evapotranspiration.

# 3 References

* DWA-A 102, 2017. Niederschlagsbedingte Siedlungsabﬂüsse – Grundsätze und Anforderungen
zum Umgang mit Regenwasser (Entwurf). DWA-Arbeitsblatt 100. Hennef: Deutsche
Vereinigung für Wasserwirtschaft, Abwasser und Abfall e. V. (in German).